In [1]:
import h5py
import gc
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, Model, load_model
from keras import optimizers
from keras import layers
from keras import regularizers

In [2]:
import h5py
import numpy as np
f1 = h5py.File('../input/electron-photon/download', 'r')
f2 = h5py.File('../input/electron-photon/download_1', 'r')

Electron_X = np.array(f1['X'])
Electron_y = np.array(f1['y'])
Parton_X = np.array(f2['X'])
Parton_y = np.array(f2['y'])
print(Electron_X.shape, Electron_y.shape, Parton_X.shape, Parton_y.shape)
All_X = np.concatenate((Electron_X, Parton_X), axis=0)
All_y = np.concatenate((Electron_y, Parton_y), axis=0)
# print(All_X.shape, All_y.shape)
rand_seed = 12
index = np.random.permutation(len(All_y))
# here the dataset is flattened
All_X, All_y = All_X[index][:,:,:,0], All_y[index]
print(All_X.shape, All_y.shape)

# clear cache to save memory
del Electron_X, Electron_y, Parton_X, Parton_y

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(All_X, All_y, test_size=0.2, random_state=12)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

del All_X, All_y

(249000, 32, 32, 2) (249000,) (249000, 32, 32, 2) (249000,)
(498000, 32, 32) (498000,)
(398400, 32, 32, 1) (99600, 32, 32, 1)
(398400,) (99600,)


In [3]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1')(X_shortcut)
    X_shortcut = layers.BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = layers.Add()([X, X_shortcut])
    X = layers.Activation('relu')(X)

    return X

In [4]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = layers.Activation('relu')(X)

    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = layers.Add()([X, X_shortcut])# SKIP Connection
    X = layers.Activation('relu')(X)

    return X

In [5]:
def ResNet(input_shape=(32, 32, 1)):

    X_input = layers.Input(input_shape)

    X = layers.ZeroPadding2D((3, 3))(X_input)

    X = layers.Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X)
    X = layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = layers.Activation('relu')(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[32, 32, 64], stage=2, block='a', s=1)
    X = identity_block(X, 3, [32, 32, 64], stage=2, block='b')

    X = layers.GlobalMaxPooling2D()(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet')

    return model

In [6]:
base_model = ResNet(input_shape=(32,32,1))
head_model = base_model.output
head_model = layers.Flatten()(head_model)
# head_model = layers.Dense(128, activation='relu', name='fc1', kernel_regularizer=regularizers.l2(0.001))(head_model)
head_model = layers.Dense(1, activation='sigmoid', name='fc2', kernel_regularizer=regularizers.l2(0.001))(head_model)
model = Model(inputs=base_model.input, outputs=head_model)
gc.collect()

2022-03-26 11:14:07.254843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 11:14:07.331644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 11:14:07.332462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-26 11:14:07.333660: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

146

In [7]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
opt_func = keras.optimizers.Adam(learning_rate=0.01)

checkpoint_filepath = 'saved_model'
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    save_weights_only=True,
    save_best_only=True)

opt_func = keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss='binary_crossentropy',
              optimizer=opt_func,
              metrics=['accuracy',
                       keras.metrics.AUC(name="auc", from_logits=True),
                      ])


history = model.fit(X_train,
          y_train,
          epochs=20,
          validation_split=0.2,
          batch_size=32,
          shuffle=False,
          callbacks=[checkpoint_callback])
gc.collect()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   3200        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

2022-03-26 11:14:15.374903: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1305477120 exceeds 10% of free system memory.
2022-03-26 11:14:16.725141: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1305477120 exceeds 10% of free system memory.
2022-03-26 11:14:17.709214: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-03-26 11:14:19.622428: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


9960/9960 [==============================] - 78s 7ms/step - loss: 0.6094 - accuracy: 0.6800 - auc: 0.7328 - val_loss: 0.8528 - val_accuracy: 0.5891 - val_auc: 0.7444
Epoch 2/20
9960/9960 [==============================] - 73s 7ms/step - loss: 0.5769 - accuracy: 0.7076 - auc: 0.7671 - val_loss: 0.8576 - val_accuracy: 0.5125 - val_auc: 0.5628
Epoch 3/20
9960/9960 [==============================] - 75s 7ms/step - loss: 0.5669 - accuracy: 0.7157 - auc: 0.7770 - val_loss: 0.5772 - val_accuracy: 0.7103 - val_auc: 0.7800
Epoch 4/20
9960/9960 [==============================] - 76s 8ms/step - loss: 0.5623 - accuracy: 0.7191 - auc: 0.7818 - val_loss: 0.8633 - val_accuracy: 0.5350 - val_auc: 0.7654
Epoch 5/20
9960/9960 [==============================] - 73s 7ms/step - loss: 0.5585 - accuracy: 0.7216 - auc: 0.7852 - val_loss: 0.7382 - val_accuracy: 0.5366 - val_auc: 0.6616
Epoch 6/20
9960/9960 [==============================] - 73s 7ms/step - loss: 0.5563 - accuracy: 0.7233 - auc: 0.7874 - val_los

1543

In [8]:
model.load_weights(checkpoint_filepath)
_, accuracy, auc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")
print(f"Test AUC: {auc}")

3113/3113 [==============================] - 15s 5ms/step - loss: 0.5516 - accuracy: 0.7269 - auc: 0.7931
Test accuracy: 0.7268574237823486
Test AUC: 0.7931223511695862


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   3200        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________